In [1]:
import pandas as pd
import numpy as np
import warnings
import math
warnings.filterwarnings("ignore")

In [2]:
# Model 1
df_perf_model_1 = pd.read_csv("C:\\Users\\DeepakArun\\Documents\\Desktop\\Robyn_202407301323_init\\pareto_aggregated.csv")
df_pred_model_1 = pd.read_csv("C:\\Users\\DeepakArun\\Documents\\Desktop\\Robyn_202407301323_init\\pareto_alldecomp_matrix.csv")

In [3]:
# # Model 2
# df_perf_model_30 = pd.read_csv("C:\\Users\\MuditKanodia\\Desktop\\TWC\Model no 30\\pareto_aggregated.csv")
# df_pred_model_30 = pd.read_csv("C:\\Users\\MuditKanodia\\Desktop\\TWC\Model no 30\\pareto_alldecomp_matrix.csv")

In [3]:
def mape(actual, pred):
    return np.mean(abs(actual - pred) / actual)

def SMAPE(actual, pred):
    smape = abs(actual-pred)/(actual + pred)
    smape = np.mean(smape[~smape.isna()])
    return smape

def cal_mape_smape(metric, actual_, pred_, train_size, val_size, test_size):
    result = []
    result =[]
    for split in ['train', 'val', 'test']:
        if split == 'train':
            actual = actual_[0:train_size]
            pred = pred_[0:train_size]
        elif split == 'val':
            actual = actual_[train_size:val_size+train_size]
            pred = pred_[train_size:val_size+train_size]
        else:
            actual = actual_[val_size+train_size:val_size+train_size+test_size]
            pred = pred_[val_size+train_size:val_size+train_size+test_size]
        
#         print(len(actual), len(pred))
#         print(actual, pred)
        
        if metric == 'MAPE':
            result = result + [mape(actual, pred)]
        else:
            result = result + [SMAPE(actual, pred)]
#     print(result)
    return result
    
def model_performance_summary(df_perf, df_actual_vs_fitted, modelID=None):
    
    if modelID == None:
        model_itr_list = list(df_perf['solID'].unique())
    else:
        model_itr_list = [modelID]
    
    output_df = pd.DataFrame(columns= [
        'solID',
        'rn',
        'spend_share',
        'effect_share',
        'coef',
        'coef_rank',
        'xDecompMeanNon0Perc',
        'rsq_train',
        'rsq_val',
        'rsq_test',
        'nrmse_train',
        'nrmse_val',
        'nrmse_test',
        'decomp.rssd',
        'mape',
        'mape_train',
        'mape_val',
        'mape_test',
        'smape_train',
        'smape_val',
        'smape_test',
        'top_sol'
    ])
    
    var_list = [
        'rn',
        'coef',
        'xDecompMeanNon0Perc',
        'rsq_train',
        'rsq_val',
        'rsq_test',
        'nrmse_train',
        'nrmse_val',
        'nrmse_test',
        'decomp.rssd',
        'mape',
        'solID',
        'spend_share',
        'effect_share',
        'top_sol'
    ]
    
    for modelID in model_itr_list:
#         print(modelID)
        df_perf_temp = df_perf[df_perf['solID']==modelID].reset_index(drop=True)
        df_actual_vs_fitted_temp = df_actual_vs_fitted[df_actual_vs_fitted['solID']==modelID].reset_index(drop=True)
        train_size = round(df_perf_temp['train_size'][0]*df_actual_vs_fitted_temp.shape[0])
        val_size = round((df_actual_vs_fitted_temp.shape[0]-train_size)/2)
        test_size = round(df_actual_vs_fitted_temp.shape[0]-train_size-val_size)
#         print(train_size, val_size, test_size)
        df_perf_temp = df_perf_temp[var_list]
        df_perf_temp['coef_rank'] =  df_perf_temp['coef'].rank(ascending=False, method='dense')
        mape_res = cal_mape_smape('MAPE', df_actual_vs_fitted_temp['dep_var'], df_actual_vs_fitted_temp['depVarHat'], train_size, val_size, test_size)
        df_perf_temp['mape_train'] = mape_res[0]
        df_perf_temp['mape_val'] = mape_res[1]
        df_perf_temp['mape_test'] = mape_res[2]
        smape_res = cal_mape_smape('SMAPE', df_actual_vs_fitted_temp['dep_var'], df_actual_vs_fitted_temp['depVarHat'], train_size, val_size, test_size)
        df_perf_temp['smape_train'] = smape_res[0]
        df_perf_temp['smape_val'] = smape_res[1]
        df_perf_temp['smape_test'] = smape_res[2]
        output_df = pd.concat([output_df, df_perf_temp])

    return output_df

In [4]:
output_df = model_performance_summary(df_perf_model_1, df_pred_model_1)

In [5]:
output_df

,solID,rn,spend_share,effect_share,coef,coef_rank,xDecompMeanNon0Perc,rsq_train,rsq_val,rsq_test,...,nrmse_test,decomp.rssd,mape,mape_train,mape_val,mape_test,smape_train,smape_val,smape_test,top_sol
0,1_1021_2,(Intercept),NaN,NaN,0.000000e+00,23.0,0.000000,0.732576,0.883254,0.525240,...,1.009612,0.478879,0,0.113787,0.052283,0.081564,0.055762,0.025985,0.042701,True
1,1_1021_2,trend,NaN,NaN,7.330967e-01,20.0,0.436475,0.732576,0.883254,0.525240,...,1.009612,0.478879,0,0.113787,0.052283,0.081564,0.055762,0.025985,0.042701,True
2,1_1021_2,season,NaN,NaN,3.622085e-01,21.0,-0.001449,0.732576,0.883254,0.525240,...,1.009612,0.478879,0,0.113787,0.052283,0.081564,0.055762,0.025985,0.042701,True
3,1_1021_2,holiday,NaN,NaN,3.175650e-01,22.0,-0.005989,0.732576,0.883254,0.525240,...,1.009612,0.478879,0,0.113787,0.052283,0.081564,0.055762,0.025985,0.042701,True
4,1_1021_2,Preference,NaN,NaN,1.877650e+07,2.0,0.231578,0.732576,0.883254,0.525240,...,1.009612,0.478879,0,0.113787,0.052283,0.081564,0.055762,0.025985,0.042701,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24,5_1812_7,SocialEng_Facebook_Estimated_Clicks,NaN,NaN,0.000000e+00,20.0,0.000000,0.774698,0.957373,0.285787,...,1.238294,0.518851,0,0.096955,0.030029,0.101213,0.047772,0.014960,0.054363,False
25,5_1812_7,SocialEng_LinkedIn_Impressions,NaN,NaN,2.122214e+05,14.0,0.002527,0.774698,0.957373,0.285787,...,1.238294,0.518851,0,0.096955,0.030029,0.101213,0.047772,0.014960,0.054363,False
26,5_1812_7,SocialEng_TikTok_Impressions,NaN,NaN,0.000000e+00,20.0,0.000000,0.774698,0.957373,0.285787,...,1.238294,0.518851,0,0.096955,0.030029,0.101213,0.047772,0.014960,0.054363,False
27,5_1812_7,SocialEng_YouTube_Estimated_Clicks,NaN,NaN,0.000000e+00,20.0,0.000000,0.774698,0.957373,0.285787,...,1.238294,0.518851,0,0.096955,0.030029,0.101213,0.047772,0.014960,0.054363,False


In [6]:
output_df.to_clipboard()